## Using Gradio to show chunking.


In [1]:
!pip install gradio==4.19.1
!pip install langchain==0.1.7
!pip install transformers==4.37.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.0 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=

In [2]:
### Utilities to get overlap between strings

def get_overlap_length(left: str, right: str):
    good_length, overlap = 0, ""
    for i in range(min(len(left), len(right))):
        if left[-i:] == right[:i]:
            good_length = i
            overlap = left[-i:]
    return good_length, overlap

def get_overlap_list(strings):
    """
    Returns a list of tuples of the form (overlap_length, overlap), one tuple for each pair of strings in the input list.
    """
    overlaps = []
    for i in range(len(strings) - 1):
        overlaps.append(get_overlap_length(strings[i], strings[i+1]))
    return overlaps

def unoverlap_list(strings):
    """
    Returns a list of tuples of the form (content, is_overlap), where is_overlap is a boolean indicating whether the content is an overlap or not.
    """
    overlaps = get_overlap_list(strings)
    new_list = []
    for index, string in enumerate(strings):
        # Add the last overlap when needed
        if index > 0 and len(overlaps[index-1][1]) > 0:
            new_list.append((overlaps[index-1][1], True))

        # prune the string with left and right overlaps
        left_overlap_length, right_overlap_length = 0, 0
        if index > 0:
            left_overlap_length = overlaps[index-1][0]
        if index < len(strings) - 1:
            right_overlap_length = overlaps[index][0]

        new_list.append((string[left_overlap_length:len(string)-right_overlap_length], False))
    return new_list

In [3]:
import gradio as gr
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    Language,
)
from transformers import AutoTokenizer
#from overlap import unoverlap_lis

In [4]:
LABEL_TEXTSPLITTER = "🦜🔗 LangChain's CharacterTextSplitter"
LABEL_RECURSIVE = "🦜🔗 LangChain's RecursiveCharacterTextSplitter"

bert_tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

def length_tokens(txt):
    return len(bert_tokenizer.tokenize(txt))


def extract_separators_from_string(separators_str):
    try:
        separators_str = separators_str.replace("\\n", "\n").replace("\\t", "\t").replace("\\\\", "\\") # fix special characters
        separators = separators_str[1:-1].split(", ")
        return [separator.replace('"', "").replace("'", "") for separator in separators]
    except Exception as e:
        raise gr.Error(f"""
        Did not succeed in extracting seperators from string: {separator_str} due to: {str(e)}.
        Please type it in the correct format: "['separator_1', 'separator_2', ...]"
        """)

def change_split_selection(split_selection):
    return (
        gr.Textbox.update(visible=(split_selection==LABEL_RECURSIVE)),
        gr.Radio.update(visible=(split_selection==LABEL_RECURSIVE)),
    )

def chunk(text, length, splitter_selection, separators_str, length_unit_selection, chunk_overlap):
    separators = extract_separators_from_string(separators_str)
    length_function = (length_tokens if "token" in length_unit_selection.lower() else len)
    if splitter_selection == LABEL_TEXTSPLITTER:
        text_splitter = CharacterTextSplitter(
            chunk_size=length,
            chunk_overlap=int(chunk_overlap),
            length_function=length_function,
            strip_whitespace=False,
            is_separator_regex=False,
            separator=" ",
        )
    elif splitter_selection == LABEL_RECURSIVE:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=length,
            chunk_overlap=int(chunk_overlap),
            length_function=length_function,
            strip_whitespace=False,
            separators=separators,
        )
    splits = text_splitter.create_documents([text])
    text_splits = [split.page_content for split in splits]
    unoverlapped_text_splits = unoverlap_list(text_splits)
    output = [((split[0], 'Overlap') if split[1] else (split[0], f"Chunk {str(i)}")) for i, split in enumerate(unoverlapped_text_splits)]
    return output

def change_preset_separators(choice):
    text_splitter = RecursiveCharacterTextSplitter()
    if choice == "Default":
        return ["\n\n", "\n", " ", ""]
    elif choice == "Markdown":
        return text_splitter.get_separators_for_language(Language.MARKDOWN)
    elif choice == "Python":
        return text_splitter.get_separators_for_language(Language.PYTHON)
    else:
        raise gr.Error("Choice of preset not recognized.")


EXAMPLE_TEXT = """
Deming’s Journey to Profound Knowledge - How Deming Helped Win a War, Altered the Face of Industry, and Holds the Key to Our Future - Part 1 - Chapter 1: Humble Origins and Non-Determinism

Deming’s Humble Beginnings

Deming’s one childhood claim to fame was when Buffalo Bill recognized him in the crowd during a performance of “Buffalo Bill’s Wild West Show” outside Los Angeles where “Edwards,” as his family called him, was visiting his cousins.

The notoriously flamboyant showman knew the boy from Cody, Wyoming, by sight if not by name. Buffalo Bill was arguably the most world-famous living American at the time, having extensively toured the US and then Europe, performing before Queen Victoria herself twice. His act made him not only famous but rich. But Buffalo Bill wanted to be taken seriously as a legitimate businessman instead of a circus act. He used his money and influence in an enterprise to create the largest undertaking of its kind ever attempted in the West: using irrigation to create an agricultural empire stretching along the Shoshone River. He began by incorporating a small town in 1896 about fifty miles east of Yellowstone National Park, which Colonel William Frederick “Buffalo Bill” Cody humbly named after himself. Life in and around Cody, Wyoming, revolved around Bill, and Bill was usually in and around “the sweetest hotel that ever was.”1

Named after his daughter, the Irma Hotel not only housed travelers but served as Bill’s headquarters, comprising two personal suites and his professional offices. His hotel was, in effect, the heart of his endeavor. When he built it, he envisaged something akin to an African city serving as a staging point for safari expeditions, perhaps like Zanzibar or Mombasa. He foresaw hosting European nobility hunting big game, East Coast financiers looking for potential investments, and opportunists from everywhere investigating the mining and ranching possibilities. The bread and butter of the Irma, however, would be the promised flood of newcomers flocking to settle the soon-to-be-verdant plains around Cody.

Shortly after the Irma opened in 1902, the Demings arrived from Sioux City, Iowa. The thirty-three-year-old father, William, had been trained as a law clerk but now sought to make his fortune on the frontier. He arranged for temporary employment with an attorney in Cody, then moved his wife and two toddling boys from the breadbasket of America to the barren badlands of Wyoming. The town was still in its infancy. There wasn’t enough legal work to keep the young father employed full time, so he found a job at the Irma as a sort of jack-of-all-trades. In addition to his wages, the hotel provided him and his family with a small house on the grounds. Edwards and his brother became regular fixtures at the Irma. Thus, Buffalo Bill recognized Edwards and his little brother at the LA performance.

Unfortunately for Buffalo Bill, the Cody-based irrigation empire failed. But in 1905, the federal government began a massive public works project via the US Reclamation Service aimed at irrigating ninety thousand acres to turn the semi-arid Bighorn Basin into fertile farms. This necessitated the construction of the Shoshone River Dam twenty-five miles northeast of Cody, around the settlement of Powell. Once completed, the concrete-arch gravity dam—itself a predecessor to the Hoover Dam—was the tallest dam in the world.

The area around Powell was opened to homesteaders, and, in 1906, Mr. William Deming applied for and received forty acres of farmland on the edge of town. Or at least what everyone hoped would be farmland one day. In the meantime, the Demings eked out what living they could out where the Great Plains meet the Rocky Mountains.

Decades later, Ed recalled his family’s hardscrabble life. “Our house in Powell, roughly 1908 to 1912, was a tarpaper shack about the size of a freight car. . . . Electricity and indoor plumbing were out of the question. Snow blew in through the cracks in the door and in the windows.” He recollected owning a cat at the time that slept with him and his brother, keeping them warm at night.

The Shoshone River Dam (also known as Buffalo Bill Dam) was completed in 1910, but with or without irrigation, Deming’s father was never a successful farmer. He once remarked, “A farmer makes his money on the farm and spends it in town. An agriculturalist makes his money in town and spends it on the farm . . . . I’m an agriculturalist.” The eggs, milk, and vegetables that kept the family alive came from their chickens, cow, and garden. Despite their efforts, it wasn’t always enough. Late in his life, Dr. Deming would recall his childhood. “I remember my mother, tak-
ing my brother and me by the hand, prayed for food.”

Elizabeth, his younger sister and the first baby born in Powell, later noted, “We didn’t have much, but nobody had anything.” To make ends meet, William continued to do some legal work in the area while Mrs. Pluma Deming, neé Edwards, taught piano and voice lessons on her Steinway parlor grand piano. William later began traveling, selling real estate and insurance. Over time, his business grew enough that the Demings were able to move out of the little tar shack on the prairie and into a slightly better home.

Though poor, his parents were well educated and poured their knowledge (and, perhaps, thirst for more) into their children. Edwards was raised in an atmosphere that included both left-brain and right-brain learning—his mother provided the right-brain perspective— the synthetic and creative aspects of learning . . . through music, while his father tended more toward the left-brain (cognitive) perspective. The atmosphere created by Deming’s parents served as the basis for his intellectual achievements and quite likely spurred the qualities which contributed to his success—an intense work ethic, devotion to spouse and family, a love of music. This environment set the course for Deming’s life. His hard-knocks upbringing gave him a unique perspective on the on-the-ground reality of the working class that managers—especially those from more privileged backgrounds—couldn’t appreciate. And the atmosphere his parents fostered helped him became a philomath: a lifelong learner and boundary spanner, a master of engineering and statistics in addition to a musician, composer, and linguist. This all resulted in a young man who was serious, studious, and diligent. The family even came to (prophetically) nickname him “the Professor.”

Newton’s Apple and Schrödinger’s Cat Set the Stage The year before the Demings moved from the Irma to the tar shack in Powell, a theoretical physicist published four groundbreaking papers that challenged the laws of physics (and would be instrumental in framing much of the thinking in Deming’s Profound Knowledge). One of the papers held the seeds of what would become the world’s most famous formula, E=mc2.

Before Einstein, everyone relied on Sir Isaac Newton’s explanations of how the physical world works (e.g., for every action there’s an equal and opposite reaction, objects in motion tend to stay in motion unless acted upon by an outside force, etc.). Einstein showed the world that physics is more like the Korean DMZ: Newton’s laws applied only in certain jurisdictions. Past that border was a whole other world. That border was the atom, the basic building block of all matter. Newtonian physics governed the apple: If you drop it, it will fall to the ground. However, once you get to the quantum level, everything goes squirrely. You can’t be certain what subatomic particles will do. For example, from the subatomic perspective, if you drop an apple, it may or may not hit the ground. It’s enough to make your head hurt.

Others quickly built on Einstein’s work throughout the 1920s, including Niels Bohr (who later worked on the Manhattan Project) and Erwin Schrödinger. But these physicists’ discoveries were just pieces of a much bigger shift: the rise of non-determinism.

Before Einstein published his famous E=mc2, and his following Nobel prize for his discovery of the law of the photoelectric effect, the world was seen through the lens of determinism (Newtonian Physics): “If I drop this apple, it will fall.” In simpler terms, the world operates solely on cause and effect. Take the weather, for instance. Decades ago, meteorologists believed that if they knew all the variables, such as humidity, wind direction, barometric pressure, etc., they could predict the weather with 100% accuracy. In fact, the earliest computers were created expressly for calculating all these variables. But even with the advances in technology we have today, meteorologists are never spot-on all the time. Even if we can calculate every variable, there’s still randomness, non-determinism.

Non-determinism also has roots in Charles Darwin’s Theory of Evolution: If you cross a black cow with a black cow, the offspring will probably be another black cow . . . but a gene might mutate and result in a two-headed white calf. You just can’t ever be certain. This is what physicist Max Planck (the father of quantum mechanics), Einstein, and others observed: No matter how much you know, there is an infinite amount of chance and randomness in the universe. Therefore, there can be no such thing as absolute certainty; the world is constantly in flux. This academic environment prepared Deming to pursue probability and statistics, a cornerstone to his Theory of Variation.

Niels Bohr and Werner Heisenberg took this idea of infinite variability to its extreme with the Copenhagen interpretation, which states that a quantum particle does not exist in one state or another, but in all of its possible states at the same time.

Erwin Schrödinger gave us an easy way to understand how those two physicists saw the way the world works (the difference between deterministic thinking and non-deterministic thinking). Say you put a cat in a sealed box. Inside, there are two items. One is a can of poisonous gas. The other is a radioactive isotope giving off gamma rays. When the isotope decays and releases gamma rays, it triggers the poison gas. The cat dies. The catch is you can’t predict when the isotope will decay.

Radioactive decay can be random; no two isotopes decay at the same rate. It could decay in a minute or in a thousand years. Therefore, you’ll never know when the isotope in the box will decay, triggering the poisonous gas. According to Bohr and Heisenberg, since you can’t predict when the element will decay, you can never be sure at any given moment whether the cat is dead or alive. Until you open the box to see for yourself, you have to simultaneously assume that the cat is alive and that it’s dead. Schrödinger’s thought experiment here was to show the absurdity of those physicists’ extreme and extremely theoretical view.

While this cat-in-the-box concept is funny, it illustrates how these two schools of thought differed. Determinism saw the world in black and white, cause and effect. With enough information, you could control any situation.

Non-determinism sees the world in shades of gray. Everything has an element of randomness. Much of how the world works is unknowable. Mathematical formulas don’t always hold true; we can’t accurately predict the future. We can only speak in probabilities: “The apple will more than likely hit the ground, but we can’t say that with 100% certainty.”

This idea of non-determinism—that reality is inherently random—would form the basis of Deming’s worldview when he began his academic career. It taught him to see the world as a series of interconnected systems, sparking the beginning of his questioning knowledge and leading to the first element of the System of Profound Knowledge: How do we know what we know?

Missing the Forest for the Trees

Let’s look at a real-life example of non-determinism.

Post–World War II, the island of Borneo in Southeast Asia had a serious malaria problem. In 1952, the World Health Organization (WHO) of the newly formed United Nations sent antimalarial experts to address the situation. One of the primary carriers of malaria is mosquitoes. Over the next three years, the WHO sprayed the chemical pesticide DDT on interior surfaces in the village longhouses, each of which housed about a hundred families. After malaria cases sharply declined, the WHO declared the mission accomplished and proceeded to host a world assembly in Mexico City to extol the virtues of DDT.

Five years after the conference, Borneo started raining cats. Literally.

And not just any cats. These were special cats: twenty-three rat catchers that floated down in their very own little cat parachutes from a British Royal Air Force transport plane.

The cats’ mission: to replenish the island’s feline population. What happened to the native cats? As it turns out, DDT had killed more than mosquitoes. Later autopsies revealed that the WHO’s practices resulted in lethal amounts of DDT accumulating in cats. Without their natural predator, the rat population exploded. Rats don’t just eat crops; they carry diseases. In Borneo’s case, typhus and sylvatic plague (the same bacterium that caused the bubonic plague of Black Plague fame). Nature could reset the ratio of cats to rats elsewhere, perhaps, but Borneo is an island. If all the cats die, there are no more cats. To remedy the WHO’s mistake, the RAF flew twenty-three cats (plus three tons of food and supplies), blessed them to “go forth and multiply,” and let ’er rip.

I imagine it was a carnival of carnivorous and carnal delight. The good folks at the WHO made an honest mistake. After all, the Western education system stresses analytical, deterministic thinking. In this case, it led to this line of reasoning:

1. Malaria is bad in Borneo.
2. Malaria is carried by mosquitoes.
3. DDT kills mosquitoes.
4. Therefore, we should use DDT to kill the mosquitoes in Borneo.

Cats killing rats and therefore keeping typhus at manageable levels is what Donella Meadows, in Thinking in Systems, calls a balancing feedback loop. However, when the cat population was out of balance, the natural order of things oscillated, creating what she describes as an overshoot of a reinforcing feed-back loop.

If the WHO had embraced non-deterministic thinking, they would have taken a much wider view of the problem. The opposite of analytic thinking is systems thinking (a.k.a. appreciation of a system): the ability to see how one thing is part of a larger, connected system. Someone who approached the ecosystem as a system might have thought along these lines:

1. Malaria is bad in Borneo.
2. Malaria is carried by mosquitoes.
3. DDT kills mosquitoes . . . but what else could it kill?
4. What else would spraying DDT on the inside of longhouses affect?
5. Do we have enough information to make an overall decision?
6. We should hold back until we can be reasonably sure we’re going to make things better and not worse for the people of Borneo.

The WHO focused only on the immediate problem and failed to consider how one “solution” might trigger a chain reaction. They failed to see the whole system. This is exactly what I meant earlier about Profound Knowledge: profound change requires Profound Knowledge, and one of the tenets of Profound Knowledge is systems thinking, an ability to see the situation in its greater context.

Determinism and analytical thinking break down a problem into tiny pieces, whereas non-determinism and systems thinking look at a problem’s bigger picture.

Analytical thinkers say, “Mission accomplished. Now, let’s go home.”

Systems thinkers say, “What were the results? Now, let’s make it even
better.”

This was bleeding-edge thinking when a sixteen-year-old’s train rolled into Laramie, Wyoming. “The Professor” was going to college.

The Professor Becomes the Student

Ed, as he would come to introduce himself, was used to shouldering a heavy load. He expected things to be no different at the University of Wyoming. In fact, he decided to major in electrical engineering. Electricity at the time was still at the forefront of technological progress, so this was like majoring in artificial intelligence or quantum computing today.

As he studied electrical engineering over the next four years, he supported himself by working as a janitor, shoveling snow, and cutting ice. He also cut railroad ties and worked at a dry cleaner. At some point, he was a soda jerk serving up malted milkshakes. On top of working and studying, he also sang in a church choir and played the piccolo in the university’s marching band. This blue-collar work ethic as well as his continued pursuit of service and the arts were a recurring pattern throughout his life. Ed was self-sufficient yet always found time to help those in need.

Ed graduated in four years but stayed for a fifth to study mathematics before enrolling at the University of Colorado for a master’s in physics and mathematics. After he graduated with his master’s in 1924, one of his instructors encouraged him to continue with his studies, perhaps at Yale. He moved to New Haven, where, three years later, he would earn his PhD in mathematical physics, the basis of probabilities and statistics and the backbone of non-deterministic thinking.

The 1920s were an exciting time for scientific discovery. The year Ed received his PhD, the fifth Solvay Conference on Physics was held. The subject was electrons and photons. In attendance were some of the most famous names in science to this day, including Marie Curie, Edwin Schrödinger, Max Planck, Albert Einstein, Niels Bohr, and Werner Heisenberg. The conference spawned an explosion in scientific thought and discovery based on non-determinism.

Non-determinism played a crucial role in shaping Deming’s worldview and began to lay the foundations for his System of Profound Knowledge. For one, it taught him that long-established and long-held beliefs weren’t necessarily true; the entire structure of the physical world was being rethought and reexamined.

Second, it showed him that the underpinnings of our very existence are random. That idea of randomness would be born out through his fascination with statistics, which in turn would inform his understanding of variation (the second element in the System of Profound Knowledge).

Third, it taught him to look beyond black-and-white cause and effect. It forced him to look at problems as multifaceted, complex systems, where changing one factor might have far-reaching, and unintended, consequences. This was the beginnings of his understanding of the fourth element of Profound Knowledge: Systems Thinking.

But before we go forward, we need to rewind briefly. During the two summers bracketing Yale’s academic calendar, Ed Deming—a university faculty member with a bachelor’s in engineering, a master’s in physics and was working on a PhD—supported himself and his wife (he’d married a schoolteacher, Agnes, in 1923), as ever, by working.

Then, Ed took an internship in a Chicago sweatshop: Hawthorne Works.

"""


with gr.Blocks(theme=gr.themes.Soft(text_size='lg', font=["monospace"], primary_hue=gr.themes.colors.green)) as demo:
    text = gr.Textbox(label="Your text 🪶", value=EXAMPLE_TEXT)
    with gr.Row():
        split_selection = gr.Dropdown(
            choices=[
                LABEL_TEXTSPLITTER,
                LABEL_RECURSIVE,
            ],
            value=LABEL_RECURSIVE,
            label="Method to split chunks 🍞",
        )
        separators_selection = gr.Textbox(
            elem_id="textbox_id",
            value=["\n\n", "\n", " ", ""],
            info="Separators used in RecursiveCharacterTextSplitter",
            show_label=False, # or set label to an empty string if you want to keep its space
            visible=True,
        )
        separator_preset_selection = gr.Radio(
            ['Default', 'Python', 'Markdown'],
            label="Choose a preset",
            info="This will apply a specific set of separators to RecursiveCharacterTextSplitter.",
            visible=True,
        )
    with gr.Row():
        length_unit_selection = gr.Dropdown(
            choices=[
                "Character count",
                "Token count (BERT tokens)",
            ],
            value="Character count",
            label="Length function",
            info="How should we measure our chunk lengths?",
        )
        slider_count = gr.Slider(
            50, 500, value=200, step=1, label="Chunk length 📏", info="In the chosen unit."
        )
        chunk_overlap = gr.Slider(
            0, 50, value=10, step=1, label="Overlap between chunks", info="In the chosen unit."
        )
    out = gr.HighlightedText(
        label="Output",
        show_legend=True,
        show_label=False,
        color_map={'Overlap': '#DADADA'}
    )

    split_selection.change(
        fn=change_split_selection,
        inputs=split_selection,
        outputs=[separators_selection, separator_preset_selection],
    )
    separator_preset_selection.change(
        fn=change_preset_separators,
        inputs=separator_preset_selection,
        outputs=separators_selection,
    )
    gr.on(
        [text.change, length_unit_selection.change, separators_selection.change, split_selection.change, slider_count.change, chunk_overlap.change],
        chunk,
        [text, slider_count, split_selection, separators_selection, length_unit_selection, chunk_overlap],
        outputs=out
    )
    demo.load(chunk, inputs=[text, slider_count, split_selection, separators_selection, length_unit_selection, chunk_overlap], outputs=out)
demo.launch()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ec3814af7da09d7bed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


That's all! Go ahead and open that share link in a new tab. Check out our [getting started](https://gradio.app/getting_started.html) page for more complicated demos.